In [2]:
import numpy as np
import pandas as pd

**generate rolling window**

In [ ]:
def generate_rolling_windows(data, total_years, initial_train_size, val_size):
    total_years = total_years
    windows = []
    initial_train_size = initial_train_size
    val_size = val_size

    # Loop through the years to create rolling windows
    for i in range(initial_train_size + 1, total_years-val_size):
        idx_1 = '20' + str(i).zfill(2) + '-01-01'
        idx_2 = '20' + str(i + val_size).zfill(2) + '-01-01'
        
        training = data[:idx_1].dropna()
        validation = data[idx_1:idx_2]
        testing = data[idx_2:].dropna()
        
        train_com = data[:idx_2].dropna()

        windows.append((training, validation, testing, train_com))
        
    return windows

**R^2-oos Enet**

In [ ]:

def r2_score_wo_demeaning(y_true, y_pred):
    y_true = np.array(y_true)
    
    ss_res = 0
    ss_tot = 0
    for i in range(len(y_true)):
        ss_res = ss_res + ((y_true[i] - y_pred[i])**2)
        ss_tot = ss_tot + ((y_true[i] - 0)**2)
    r2 = 1 - (ss_res / ss_tot)
    return float(r2)

**Tuning Enet**

In [ ]:
def tuning_1 (model, param_combos, X_train, y_train, X_val, y_val):
#   param_combos: list of dictionaries of parameters combo
    
    results = {'param': [], 'score': []}
    opt_param = 0
    opt_score = 0

    for params in param_combos:

        for param_name, param_value in params.items():
            setattr(model, param_name, param_value)

        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)

#         print("Shape of y_val:", y_val.shape)
#         print("Shape of y_pred:", y_pred.shape)
        
        r2 = r2_score_wo_demeaning(y_val, y_pred)

        results['param'].append(params)
        results['score'].append(r2)
    
    sorted_zipped = sorted(zip(results['param'], results['score']), key=lambda x: x[1])
    opt_param = sorted_zipped[0][0]
    opt_score = sorted_zipped[0][1]
    
    return opt_param, opt_score

**R^2-oos NN**

In [ ]:
import keras
from keras import layers
from keras import Sequential

In [ ]:
import tensorflow as tf

def r2_score_wo_demeaning_nn(y_true, y_pred):
    ss_res = tf.reduce_sum(tf.square(y_true - y_pred))
    ss_tot = tf.reduce_sum(tf.square(y_true - 0))
    r2 = 1 - (ss_res / ss_tot)
    return r2

**Compiling and tuning NN**

In [ ]:

def compile_and_tune_model(model, parameter_dicts, x_train, y_train, x_val, y_val):
    results = []
    
    for params in parameter_dicts:

        model.compile(optimizer=keras.optimizers.Adam(learning_rate=params['learning_rate']),
                      loss='mean_squared_error', metrics=r2_score_wo_demeaning_nn)  # Using mean absolute error (mae) as metric
        
        history = model.fit(x_train, y_train, epochs=params['epoch'], batch_size=params['batch_size'],
                            validation_data=(x_val, y_val), verbose=0)
        
        # Get the metric value for the last epoch
        last_epoch_metric = history.history['r2_score_wo_demeaning_nn'][-1]  # Validation MAE for last epoch
        
        # Store results for current parameter set
        results.append({'params': params, 'val_r2_score_wo_demeaning_nn': last_epoch_metric})
    
    return results